In [7]:
import numpy as np

In [8]:
def calculate_profit(x1,x2,y,z,n0,p1,p2,p_product):
    #参数说明
    #n:零件1的数量/零件2的数量
    #p1:零件1的次品率
    #p2:零件2的次品率
    #p_product:成品的次品率
    #一些内置的参数
    C1=2#检测零件1的成本
    C2=3#检测零件2的成本
    C_market=56#单件成品的市场售价
    C_assemble=6#装配单件成品的费用
    C_test_product=3#检测单件成品所需的费用
    C_lose=6#调换一件产品引起的损失
    C_disassemble=40#拆解一件产品所需的费用
    #更新次品率
    p1_update=(1-x1)*p1
    p2_update=(1-x2)*p2
    #成品的总次品率
    P=(1-p1_update)*(1-p2_update)*p_product+p1_update+p2_update-p1_update*p2_update
    n=np.floor(min((1-x1*p1)*n0,(1-x2*p2)*n0))
    profit=C_market*(1-P)*n-x1*C1*n0-x2*C2*n0-C_assemble*n-y*C_test_product*n-(1-y)*C_lose*n*P-z*C_disassemble*n*P
    #更新零件数
    if z==0:
        n_update=0
    elif z==1:
        n_update=np.floor(n*P)
    return profit,n_update,p1_update,p2_update

In [9]:
n0=10000
p1=0.05
p2=0.05
p_product=0.05
S=0
same=0
different=0
for p1 in np.linspace(0.95*p1,1.05*p1,5):
    for p2 in np.linspace(0.95*p2,1.05*p2,5):
        for p_product in np.linspace(0.95*p_product,1.05*p_product,5):
            S+=1
            #第一阶段的生产决策
            result=[]
            for x1 in [0,1]:
                for x2 in [0,1]:
                    for y in [0,1]:
                        for z in [0,1]:
                            profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,n0,p1,p2,p_product)
                            result.append([x1,x2,y,z,profit,n_update,p1_update,p2_update])
            #第二阶段的决策
            result1=np.empty((8, 16), dtype=object)
            for i in range(1,16,2):
                count=0
                for x1 in [0,1]:
                    for x2 in [0,1]:
                        for y in [0,1]:
                            for z in [0,1]:
                                profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result[i][5],result[i][6],result[i][7],p_product)
                                result1[int((i-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                count+=1
            #第三阶段决策
            result2=np.empty((8,8,16), dtype=object)
            for i in range(8):
                for j in range(1,16,2):
                    count=0
                    for x1 in [0,1]:
                        for x2 in [0,1]:
                            for y in [0,1]:
                                for z in [0,1]:
                                    profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result1[i,j][5],result1[i,j][6],result1[i,j][7],p_product)
                                    result2[i,int((j-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                    count+=1
            result2_value=np.zeros((8,8,16))
            for i in range(8):
                for j in range(8):
                    for k in range(16):
                        result2_value[i,j,k]=result2[i,j,k][4]
            result2_value_max=np.max(result2_value,axis=2)
            result1_value=np.zeros((8,16))
            for i in range(8):
                for j in range(16):
                    result1_value[i,j]=result1[i,j][4]
                    result1_value[:,1::2]=result1_value[:,1::2]+result2_value_max
            result1_value_max=np.max(result1_value,axis=1)
            final_result=[0 for i in range(16)]
            for i in range(16):
                final_result[i]=result[i][4]
                final_result[1::2]=final_result[1::2]+result1_value_max
            max_value=max(final_result)
            max_index = final_result.index(max_value)
            print(S)
            print(f"最终的最大利润为{max_value}")
            print("第一阶段的生产决策为:")
            print(result[max_index])
            if max_index%2:
                max_index_1 = np.argmax(result1_value[int((max_index-1)/2)])
                print("第二阶段的生产决策为:")
                print(result1[int((max_index-1)/2),max_index_1])
            if max_index_1%2:
                max_index_2 = np.argmax(result2_value[int((max_index-1)/2),int((max_index_1-1)/2)])
                print("第三阶段的生产决策为:")
                print(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2])
            if (result[max_index][0:4]==[0,0,0,1])&(result1[int((max_index-1)/2),max_index_1][0:4]==[0,0,0,1])&(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2][0:4]==[0,0,0,0]):
                same+=1
            else:
                different+=1

1
最终的最大利润为15671634.701612165
第一阶段的生产决策为:
[0, 0, 0, 1, 361444.8096875, 1358.0, 0.0475, 0.0475]
第二阶段的生产决策为:
[0, 0, 0, 1, 49084.2051555625, 184.0, 0.0475, 0.0475]
第三阶段的生产决策为:
[0, 0, 0, 0, 7650.355283249999, 0, 0.0475, 0.0475]
2
最终的最大利润为15886637.0482339
第一阶段的生产决策为:
[0, 0, 0, 1, 360288.05796874996, 1369.0, 0.0475, 0.0475]
第二阶段的生产决策为:
[0, 0, 0, 1, 49323.43513592188, 187.0, 0.0475, 0.0475]
第三阶段的生产决策为:
[0, 0, 0, 0, 7761.940925578125, 0, 0.0475, 0.0475]
3
最终的最大利润为16101336.248740653
第一阶段的生产决策为:
[0, 0, 0, 1, 359131.30625, 1381.0, 0.0475, 0.0475]
第二阶段的生产决策为:
[0, 0, 0, 1, 49596.03339312499, 190.0, 0.0475, 0.0475]
第三阶段的生产决策为:
[0, 0, 0, 0, 7873.10469375, 0, 0.0475, 0.0475]
4
最终的最大利润为16314653.174085993
第一阶段的生产决策为:
[0, 0, 0, 1, 357974.55453125003, 1392.0, 0.0475, 0.0475]
第二阶段的生产决策为:
[0, 0, 0, 1, 49830.05799075001, 193.0, 0.0475, 0.0475]
第三阶段的生产决策为:
[0, 0, 0, 0, 7983.846587765626, 0, 0.0475, 0.0475]
5
最终的最大利润为16527128.256357018
第一阶段的生产决策为:
[0, 0, 0, 1, 356817.8028124999, 1403.0, 0.0475, 0.0475]
第二阶段的生产决

In [10]:
print(f"在0.95-1.05波动时,生产决策方案和原方案一致的次数为:{same}")
print(f"在0.95-1.05波动时,生产决策方案和原方案不一致的次数为:{different}")
print(f"在0.95-1.05波动时,决策差异率为:{different/125}")

在0.95-1.05波动时,生产决策方案和原方案一致的次数为:125
在0.95-1.05波动时,生产决策方案和原方案不一致的次数为:0
在0.95-1.05波动时,决策差异率为:0.0


In [11]:
n0=10000
p1=0.05
p2=0.05
p_product=0.05
S=0
same=0
different=0
for p1 in np.linspace(0.9*p1,1.1*p1,5):
    for p2 in np.linspace(0.9*p2,1.1*p2,5):
        for p_product in np.linspace(0.9*p_product,1.1*p_product,5):
            S+=1
            #第一阶段的生产决策
            result=[]
            for x1 in [0,1]:
                for x2 in [0,1]:
                    for y in [0,1]:
                        for z in [0,1]:
                            profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,n0,p1,p2,p_product)
                            result.append([x1,x2,y,z,profit,n_update,p1_update,p2_update])
            #第二阶段的决策
            result1=np.empty((8, 16), dtype=object)
            for i in range(1,16,2):
                count=0
                for x1 in [0,1]:
                    for x2 in [0,1]:
                        for y in [0,1]:
                            for z in [0,1]:
                                profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result[i][5],result[i][6],result[i][7],p_product)
                                result1[int((i-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                count+=1
            #第三阶段决策
            result2=np.empty((8,8,16), dtype=object)
            for i in range(8):
                for j in range(1,16,2):
                    count=0
                    for x1 in [0,1]:
                        for x2 in [0,1]:
                            for y in [0,1]:
                                for z in [0,1]:
                                    profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result1[i,j][5],result1[i,j][6],result1[i,j][7],p_product)
                                    result2[i,int((j-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                    count+=1
            result2_value=np.zeros((8,8,16))
            for i in range(8):
                for j in range(8):
                    for k in range(16):
                        result2_value[i,j,k]=result2[i,j,k][4]
            result2_value_max=np.max(result2_value,axis=2)
            result1_value=np.zeros((8,16))
            for i in range(8):
                for j in range(16):
                    result1_value[i,j]=result1[i,j][4]
                    result1_value[:,1::2]=result1_value[:,1::2]+result2_value_max
            result1_value_max=np.max(result1_value,axis=1)
            final_result=[0 for i in range(16)]
            for i in range(16):
                final_result[i]=result[i][4]
                final_result[1::2]=final_result[1::2]+result1_value_max
            max_value=max(final_result)
            max_index = final_result.index(max_value)
            print(S)
            print(f"最终的最大利润为{max_value}")
            print("第一阶段的生产决策为:")
            print(result[max_index])
            if max_index%2:
                max_index_1 = np.argmax(result1_value[int((max_index-1)/2)])
                print("第二阶段的生产决策为:")
                print(result1[int((max_index-1)/2),max_index_1])
            if max_index_1%2:
                max_index_2 = np.argmax(result2_value[int((max_index-1)/2),int((max_index_1-1)/2)])
                print("第三阶段的生产决策为:")
                print(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2])
            if (result[max_index][0:4]==[0,0,0,1])&(result1[int((max_index-1)/2),max_index_1][0:4]==[0,0,0,1])&(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2][0:4]==[0,0,0,0]):
                same+=1
            else:
                different+=1

1
最终的最大利润为14363240.735645045
第一阶段的生产决策为:
[0, 0, 0, 1, 368403.55249999993, 1290.0, 0.045000000000000005, 0.045000000000000005]
第二阶段的生产决策为:
[0, 0, 0, 1, 47524.05827249999, 166.0, 0.045000000000000005, 0.045000000000000005]
第三阶段的生产决策为:
[0, 0, 0, 0, 6972.166041499999, 0, 0.045000000000000005, 0.045000000000000005]
2
最终的最大利润为14802247.628442455
第一阶段的生产决策为:
[0, 0, 0, 1, 366077.88875000004, 1312.0, 0.045000000000000005, 0.045000000000000005]
第二阶段的生产决策为:
[0, 0, 0, 1, 48029.419004, 172.0, 0.045000000000000005, 0.045000000000000005]
第三阶段的生产决策为:
[0, 0, 0, 0, 7199.857456499999, 0, 0.045000000000000005, 0.045000000000000005]
3
最终的最大利润为15238415.077587465
第一阶段的生产决策为:
[0, 0, 0, 1, 363752.22500000003, 1335.0, 0.045000000000000005, 0.045000000000000005]
第二阶段的生产决策为:
[0, 0, 0, 1, 48560.9220375, 178.0, 0.045000000000000005, 0.045000000000000005]
第三阶段的生产决策为:
[0, 0, 0, 0, 7425.852505000001, 0, 0.045000000000000005, 0.045000000000000005]
4
最终的最大利润为15671190.477751218
第一阶段的生产决策为:
[0, 0, 0, 1, 361426.5612499999, 

In [12]:
print(f"在0.9-1.1波动时,生产决策方案和原方案一致的次数为:{same}")
print(f"在0.9-1.1波动时,生产决策方案和原方案不一致的次数为:{different}")
print(f"在0.9-1.1波动时,决策差异率为:{different/125}")

在0.9-1.1波动时,生产决策方案和原方案一致的次数为:116
在0.9-1.1波动时,生产决策方案和原方案不一致的次数为:9
在0.9-1.1波动时,决策差异率为:0.072


In [13]:
n0=10000
p1=0.05
p2=0.05
p_product=0.05
S=0
same=0
different=0
for p1 in np.linspace(0.8*p1,1.2*p1,5):
    for p2 in np.linspace(0.8*p2,1.2*p2,5):
        for p_product in np.linspace(0.8*p_product,1.2*p_product,5):
            S+=1
            #第一阶段的生产决策
            result=[]
            for x1 in [0,1]:
                for x2 in [0,1]:
                    for y in [0,1]:
                        for z in [0,1]:
                            profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,n0,p1,p2,p_product)
                            result.append([x1,x2,y,z,profit,n_update,p1_update,p2_update])
            #第二阶段的决策
            result1=np.empty((8, 16), dtype=object)
            for i in range(1,16,2):
                count=0
                for x1 in [0,1]:
                    for x2 in [0,1]:
                        for y in [0,1]:
                            for z in [0,1]:
                                profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result[i][5],result[i][6],result[i][7],p_product)
                                result1[int((i-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                count+=1
            #第三阶段决策
            result2=np.empty((8,8,16), dtype=object)
            for i in range(8):
                for j in range(1,16,2):
                    count=0
                    for x1 in [0,1]:
                        for x2 in [0,1]:
                            for y in [0,1]:
                                for z in [0,1]:
                                    profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result1[i,j][5],result1[i,j][6],result1[i,j][7],p_product)
                                    result2[i,int((j-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                    count+=1
            result2_value=np.zeros((8,8,16))
            for i in range(8):
                for j in range(8):
                    for k in range(16):
                        result2_value[i,j,k]=result2[i,j,k][4]
            result2_value_max=np.max(result2_value,axis=2)
            result1_value=np.zeros((8,16))
            for i in range(8):
                for j in range(16):
                    result1_value[i,j]=result1[i,j][4]
                    result1_value[:,1::2]=result1_value[:,1::2]+result2_value_max
            result1_value_max=np.max(result1_value,axis=1)
            final_result=[0 for i in range(16)]
            for i in range(16):
                final_result[i]=result[i][4]
                final_result[1::2]=final_result[1::2]+result1_value_max
            max_value=max(final_result)
            max_index = final_result.index(max_value)
            print(S)
            print(f"最终的最大利润为{max_value}")
            print("第一阶段的生产决策为:")
            print(result[max_index])
            if max_index%2:
                max_index_1 = np.argmax(result1_value[int((max_index-1)/2)])
                print("第二阶段的生产决策为:")
                print(result1[int((max_index-1)/2),max_index_1])
            if max_index_1%2:
                max_index_2 = np.argmax(result2_value[int((max_index-1)/2),int((max_index_1-1)/2)])
                print("第三阶段的生产决策为:")
                print(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2])
            if (result[max_index][0:4]==[0,0,0,1])&(result1[int((max_index-1)/2),max_index_1][0:4]==[0,0,0,1])&(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2][0:4]==[0,0,0,0]):
                same+=1
            else:
                different+=1

1
最终的最大利润为11819245.306879966
第一阶段的生产决策为:
[0, 0, 0, 1, 382430.72, 1152.0, 0.04000000000000001, 0.04000000000000001]
第二阶段的生产决策为:
[0, 0, 0, 1, 44056.01894399999, 132.0, 0.04000000000000001, 0.04000000000000001]
第三阶段的生产决策为:
[0, 0, 0, 0, 5656.679423999999, 0, 0.04000000000000001, 0.04000000000000001]
2
最终的最大利润为12652656.661759969
第一阶段的生产决策为:
[0, 0, 0, 1, 377730.56, 1198.0, 0.04000000000000001, 0.04000000000000001]
第二阶段的生产决策为:
[0, 0, 0, 1, 45252.121088, 143.0, 0.04000000000000001, 0.04000000000000001]
第三阶段的生产决策为:
[0, 0, 0, 0, 6087.2148480000005, 0, 0.04000000000000001, 0.04000000000000001]
3
最终的最大利润为13473445.875200016
第一阶段的生产决策为:
[0, 0, 0, 1, 373030.4, 1244.0, 0.04000000000000001, 0.04000000000000001]
第二阶段的生产决策为:
[0, 0, 0, 1, 46404.981759999995, 154.0, 0.04000000000000001, 0.04000000000000001]
第三阶段的生产决策为:
[0, 0, 0, 0, 6511.46496, 0, 0.04000000000000001, 0.04000000000000001]
4
最终的最大利润为14361616.363519982
第一阶段的生产决策为:
[0, 0, 0, 1, 368330.23999999993, 1290.0, 0.04000000000000001, 0.040000000000000

In [14]:
print(f"在0.8-1.2波动时,生产决策方案和原方案一致的次数为:{same}")
print(f"在0.8-1.2波动时,生产决策方案和原方案不一致的次数为:{different}")
print(f"在0.8-1.2波动时,决策差异率为:{different/125}")

在0.8-1.2波动时,生产决策方案和原方案一致的次数为:62
在0.8-1.2波动时,生产决策方案和原方案不一致的次数为:63
在0.8-1.2波动时,决策差异率为:0.504
